In [1]:
import numpy as np
import re
from test_model import (
    encoder_model, 
    decoder_model, 
    num_decoder_tokens, 
    num_encoder_tokens, 
    input_features_dict, 
    target_features_dict, 
    reverse_target_features_dict, 
    max_decoder_seq_length, 
    max_encoder_seq_length
)


class ChatBot:
    """A chatbot that uses a trained encoder-decoder model for conversation."""
    
    # Potential negative responses
    negative_commands = ("no", "nope", "nah", "naw", "not a chance", "sorry")
    # Keywords for exiting the conversation
    exit_commands = ("quit", "pause", "exit", "goodbye", "bye", "later")
    
    def __init__(self):
        """Initialize the chatbot with intent patterns."""
        self.alienbabble = {
            'describe_planet_intent': r'.*\s*your planet.*',
            'answer_why_intent': r'why\sare.*',
            'cubed_intent': r'.*cube.*(\d+)'
        }
    
    def make_exit(self, reply):
        """
        Check if the user wants to exit the conversation.
        
        Args:
            reply (str): User's input message
            
        Returns:
            bool: True if user wants to exit, False otherwise
        """
        if not reply:  # Handle empty input
            return False
            
        reply_lower = reply.lower().strip()
        
        for exit_command in self.exit_commands:
            if exit_command in reply_lower:
                print("Ok, have a great day!")
                return True
        
        return False
    
    def start_chat(self):
        """Start the chat conversation with an initial prompt."""
        try:
            user_response = input("Would you like to chat? ").strip()
            
            # Handle empty response
            if not user_response:
                print("I didn't catch that. Maybe another time!")
                return
            
            # Check for negative response
            if user_response.lower() in self.negative_commands:
                print("Ok, have a great day!")
                return
            
            # Check for immediate exit
            if self.make_exit(user_response):
                return
            
            # Start the conversation
            print("Great! Let's chat. (Type 'quit', 'bye', or 'exit' to end the conversation)")
            self.chat(user_response)
            
        except KeyboardInterrupt:
            print("\n\nChat interrupted. Goodbye!")
        except Exception as e:
            print(f"\nAn error occurred: {e}")
            print("Chat ended.")
    
    def chat(self, reply):
        """
        Main chat loop that generates responses.
        
        Args:
            reply (str): Initial user message
        """
        try:
            while not self.make_exit(reply):
                # Generate response
                try:
                    bot_response = self.generate_response(reply)
                    
                    # Handle empty or invalid responses
                    if not bot_response or bot_response.strip() == "":
                        bot_response = "I'm not sure how to respond to that. Can you rephrase?"
                    
                    # Get next user input
                    reply = input(f"{bot_response}\nYou: ").strip()
                    
                    # Handle empty input
                    if not reply:
                        reply = input("I didn't catch that. Can you say that again?\nYou: ").strip()
                        if not reply:  # Still empty, exit gracefully
                            print("Ok, have a great day!")
                            break
                            
                except Exception as e:
                    print(f"Error generating response: {e}")
                    reply = input("Sorry, I had trouble understanding. Can you try again?\nYou: ").strip()
                    if not reply:
                        print("Ok, have a great day!")
                        break
                        
        except KeyboardInterrupt:
            print("\n\nChat interrupted. Goodbye!")
        except Exception as e:
            print(f"\nAn unexpected error occurred: {e}")
            print("Chat ended.")
    
    def string_to_matrix(self, user_input):
        """
        Convert user input string to a matrix representation for the model.
        
        Args:
            user_input (str): User's input message
            
        Returns:
            numpy.ndarray: Matrix representation of the input
        """
        if not user_input:
            # Return empty matrix for empty input
            return np.zeros((1, max_encoder_seq_length, num_encoder_tokens), dtype='float32')
        
        # Tokenize input
        tokens = re.findall(r"[\w']+|[^\s\w]", user_input.lower())
        
        # Initialize matrix
        user_input_matrix = np.zeros(
            (1, max_encoder_seq_length, num_encoder_tokens),
            dtype='float32'
        )
        
        # Populate matrix (limit to max_encoder_seq_length)
        for timestep, token in enumerate(tokens[:max_encoder_seq_length]):
            if token in input_features_dict:
                user_input_matrix[0, timestep, input_features_dict[token]] = 1.
        
        return user_input_matrix
    
    def generate_response(self, user_input):
        """
        Generate a chatbot response using the trained model.
        
        Args:
            user_input (str): User's input message
            
        Returns:
            str: Generated response from the chatbot
        """
        if not user_input or not user_input.strip():
            return "I didn't catch that. Could you please say something?"
        
        try:
            # Encode input
            input_matrix = self.string_to_matrix(user_input)
            states_value = encoder_model.predict(input_matrix, verbose=0)
            
            # Initialize target sequence
            target_seq = np.zeros((1, 1, num_decoder_tokens))
            
            # Check if START token exists
            if '<START>' not in target_features_dict:
                return "Sorry, I'm having technical difficulties."
            
            target_seq[0, 0, target_features_dict['<START>']] = 1.
            
            # Generate response
            chatbot_response = ''
            stop_condition = False
            max_iterations = max_decoder_seq_length + 10  # Safety limit
            iterations = 0
            
            while not stop_condition and iterations < max_iterations:
                iterations += 1
                
                # Predict next token
                output_tokens, hidden_state, cell_state = decoder_model.predict(
                    [target_seq] + states_value, 
                    verbose=0
                )
                
                # Sample token
                sampled_token_index = np.argmax(output_tokens[0, -1, :])
                
                # Check if token index is valid
                if sampled_token_index not in reverse_target_features_dict:
                    break
                    
                sampled_token = reverse_target_features_dict[sampled_token_index]
                
                # Add token to response (except START and END tokens)
                if sampled_token not in ['<START>', '<END>']:
                    chatbot_response += sampled_token + " "
                
                # Check stopping condition
                if sampled_token == '<END>' or len(chatbot_response.split()) >= max_decoder_seq_length:
                    stop_condition = True
                
                # Update target sequence
                target_seq = np.zeros((1, 1, num_decoder_tokens))
                target_seq[0, 0, sampled_token_index] = 1.
                states_value = [hidden_state, cell_state]
            
            # Clean up response
            chatbot_response = chatbot_response.strip()
            
            # Handle empty response
            if not chatbot_response:
                return "I'm not sure how to respond to that."
            
            return chatbot_response
            
        except Exception as e:
            print(f"Error in generate_response: {e}")
            return "Sorry, I'm having trouble generating a response."


# Main execution
if __name__ == "__main__":
    # Create an instance of ChatBot
    chatbot_instance = ChatBot()
    chatbot_instance.start_chat()

2025-12-21 18:45:13.274434: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-21 18:45:13.274992: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-21 18:45:13.277224: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-12-21 18:45:13.282333: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1766339113.291348  380400 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1766339113.29

Epoch 1/600


2025-12-21 18:45:15.935762: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)


5/5 ━━━━━━━━━━━━━━━━━━━━ 2s 131ms/step - accuracy: 0.0121 - loss: 2.3316 - val_accuracy: 0.0256 - val_loss: 1.8758
Epoch 2/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.0352 - loss: 2.3651 - val_accuracy: 0.0312 - val_loss: 1.8677
Epoch 3/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.0312 - loss: 2.2573 - val_accuracy: 0.0312 - val_loss: 1.8067
Epoch 4/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 71ms/step - accuracy: 0.0312 - loss: 2.0529 - val_accuracy: 0.0312 - val_loss: 1.7393
Epoch 5/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.0316 - loss: 2.1918 - val_accuracy: 0.0369 - val_loss: 1.7565
Epoch 6/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.0403 - loss: 2.0665 - val_accuracy: 0.0369 - val_loss: 1.7718
Epoch 7/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.0387 - loss: 1.8531 - val_accuracy: 0.0312 - val_loss: 1.7812
Epoch 8/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.0403 - loss: 1.9844 - val_accuracy: 0.0341 - val_loss: 1.7848
Epo

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.0455 - loss: 1.8338 - val_accuracy: 0.0369 - val_loss: 2.0477
Epoch 53/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.0469 - loss: 1.8559 - val_accuracy: 0.0341 - val_loss: 2.0490
Epoch 54/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.0449 - loss: 1.9383 - val_accuracy: 0.0284 - val_loss: 2.0367
Epoch 55/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.0396 - loss: 1.9187 - val_accuracy: 0.0312 - val_loss: 2.0387
Epoch 56/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.0482 - loss: 1.7374 - val_accuracy: 0.0199 - val_loss: 2.0458
Epoch 57/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.0356 - loss: 1.7248 - val_accuracy: 0.0312 - val_loss: 2.0466
Epoch 58/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.0424 - loss: 1.8001 - val_accuracy: 0.0341 - val_loss: 2.0522
Epoch 59/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.0447 - loss: 1.7410 - val_accuracy: 0.0312 - val_loss: 2.06

...

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.1813 - loss: 1.2213 - val_accuracy: 0.0284 - val_loss: 2.6127
Epoch 404/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.1761 - loss: 0.9926 - val_accuracy: 0.0284 - val_loss: 2.5729
Epoch 405/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.1762 - loss: 1.0440 - val_accuracy: 0.0312 - val_loss: 2.5712
Epoch 406/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.1692 - loss: 1.2150 - val_accuracy: 0.0227 - val_loss: 2.5899
Epoch 407/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.1897 - loss: 1.1273 - val_accuracy: 0.0312 - val_loss: 2.5198
Epoch 408/600
5/5 ━━━━━━━━━━━━━━━━━━━━ -1s -252677us/step - accuracy: 0.1764 - loss: 1.1276 - val_accuracy: 0.0341 - val_loss: 2.6146
Epoch 409/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - accuracy: 0.1987 - loss: 1.2593 - val_accuracy: 0.0369 - val_loss: 2.5710
Epoch 410/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 79ms/step - accuracy: 0.1826 - loss: 1.2153 - val_accuracy: 0.0312 - v

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.2232 - loss: 1.1318 - val_accuracy: 0.0227 - val_loss: 2.6358
Epoch 454/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.2163 - loss: 1.0570 - val_accuracy: 0.0199 - val_loss: 2.6694
Epoch 455/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.2214 - loss: 0.9050 - val_accuracy: 0.0312 - val_loss: 2.6928
Epoch 456/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.2004 - loss: 1.0416 - val_accuracy: 0.0256 - val_loss: 2.6442
Epoch 457/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.2153 - loss: 0.9292 - val_accuracy: 0.0227 - val_loss: 2.6531
Epoch 458/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.2229 - loss: 1.0734 - val_accuracy: 0.0199 - val_loss: 2.6800
Epoch 459/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.2162 - loss: 0.9980 - val_accuracy: 0.0057 - val_loss: 2.6797
Epoch 460/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.2116 - loss: 0.8683 - val_accuracy: 0.0227 - val_los

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.2426 - loss: 0.9345 - val_accuracy: 0.0057 - val_loss: 2.6844
Epoch 504/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 72ms/step - accuracy: 0.2227 - loss: 0.9291 - val_accuracy: 0.0312 - val_loss: 2.6655
Epoch 505/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.2413 - loss: 1.0023 - val_accuracy: 0.0227 - val_loss: 2.6965
Epoch 506/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - accuracy: 0.2230 - loss: 0.9015 - val_accuracy: 0.0114 - val_loss: 2.6825
Epoch 507/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.2178 - loss: 0.8562 - val_accuracy: 0.0085 - val_loss: 2.7104
Epoch 508/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 73ms/step - accuracy: 0.2276 - loss: 0.9495 - val_accuracy: 0.0256 - val_loss: 2.6911
Epoch 509/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.2289 - loss: 0.8198 - val_accuracy: 0.0199 - val_loss: 2.6974
Epoch 510/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.2259 - loss: 0.9218 - val_accuracy: 0.0284 - val_los

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.2375 - loss: 0.7958 - val_accuracy: 0.0369 - val_loss: 2.6391
Epoch 554/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.2605 - loss: 0.9964 - val_accuracy: 0.0256 - val_loss: 2.6798
Epoch 555/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - accuracy: 0.2328 - loss: 0.8699 - val_accuracy: 0.0256 - val_loss: 2.6660
Epoch 556/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.2371 - loss: 0.7835 - val_accuracy: 0.0227 - val_loss: 2.6268
Epoch 557/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.2544 - loss: 0.8090 - val_accuracy: 0.0256 - val_loss: 2.6067
Epoch 558/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.2531 - loss: 0.8264 - val_accuracy: 0.0341 - val_loss: 2.6054
Epoch 559/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.2557 - loss: 0.7296 - val_accuracy: 0.0256 - val_loss: 2.6150
Epoch 560/600
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 75ms/step - accuracy: 0.2539 - loss: 0.7260 - val_accuracy: 0.0312 - val_los

Would you like to chat? yes
Great! Let's chat. (Type 'quit', 'bye', or 'exit' to end the conversation)
we just love trolls
You: are you a troll?
what would you like to order ! ! ! cat ❤ ❤ ️ ️ ️
You: im changing my cats name to tofu
imagine naming ur cat gooby instead of tofu tofu begin begin . .
You: exit
Ok, have a great day!
